In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score,fbeta_score
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore',category=UndefinedMetricWarning)

In [2]:
df = pd.read_excel('sdbi_proxy_bad_def_dataset.xlsx')

In [3]:
df_copy = df.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172929 entries, 0 to 172928
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   id      172929 non-null  int64
 1   mob     172929 non-null  int64
 2   dpd     172929 non-null  int64
dtypes: int64(3)
memory usage: 4.0 MB


In [5]:
df.head()

,id,mob,dpd
0,1,0,0
1,1,1,0
2,1,2,0
3,1,3,0
4,1,4,0


In [6]:
## Unique customers

In [6]:
df.id.nunique()

6999

In [7]:
## Case 1: Some months can be missing
all_months = list(range(0,25))
rem_ids = []
for i in df.id.unique():
    if ((list(df.query('id == {}'.format(i))['mob'].values) == all_months) or (list(df.query('id == {}'.format(i))['mob'].values) == all_months[1:])):
        continue
    else:
        print('id ',i,' has some missing months')
        rem_ids.append(i)

id  5253  has some missing months
id  5292  has some missing months


In [8]:
rem_ids

[5253, 5292]

In [9]:
for i in rem_ids:
    df.drop(df[df['id'] == i].index,inplace=True)

In [10]:
grouped_df = df.groupby('id')[['dpd']].max()

In [11]:
grouped_df.rename(columns={'dpd':'max_dpd'},inplace=True)

In [12]:
grouped_df.head(15)

,max_dpd
id,
1,0
2,28
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [13]:
grouped_df['eventual_bad'] = np.where(grouped_df['max_dpd'] >= 90,1,0)

In [14]:
grouped_df.head(15)

,max_dpd,eventual_bad
id,,
1,0,0
2,28,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [15]:
len(grouped_df[grouped_df['eventual_bad'] == 1])/len(grouped_df)

0.033442904101757896

## Proxy bad defn

- Went x+ in first 12 months

In [16]:
df_ = df_copy.drop(df[df['mob'] > 12].index)

In [17]:
proxy_bad_df = df_.groupby('id')[['dpd']].max()

In [18]:
proxy_bad_df.columns

Index(['dpd'], dtype='object')

In [19]:
proxy_bad_df.rename(columns={'dpd':'max_dpd'},inplace=True)

In [20]:
proxy_bad_df['proxy_bad'] = np.where(proxy_bad_df['max_dpd']>0,1,0)

In [21]:
proxy_bad_df.head(15)

,max_dpd,proxy_bad
id,,
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [22]:
merged_df = pd.merge(grouped_df,proxy_bad_df,right_index=True,left_index=True)

In [23]:
merged_df.head(15)

,max_dpd_x,eventual_bad,max_dpd_y,proxy_bad
id,,,,
1,0,0,0,0
2,28,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [24]:
confusion_matrix(merged_df['eventual_bad'],merged_df['proxy_bad'])

array([[6513,  250],
       [ 114,  120]])

In [25]:
precision_score(merged_df['eventual_bad'],merged_df['proxy_bad'])

0.32432432432432434

In [26]:
accuracy_score(merged_df['eventual_bad'],merged_df['proxy_bad'])

0.9479777047305988

In [27]:
recall_score(merged_df['eventual_bad'],merged_df['proxy_bad'])

0.5128205128205128

#### Other proxy bad defns

- Went x+ in first 3 months
- Went 30+ in first 3 months
- Went 60+ in first 3 months
- Went 90+ in fist 3 months
- Went x+ in first 6 months
- Went 30+ in first 6 months
- Went 60+ in first 6 months
- Went 90+ in fist 6 months
- Went x+ in first 9 months
- Went 30+ in first 9 months
- Went 60+ in first 9 months
- Went 90+ in fist 9 months
- Went x+ in first 12 months
- Went 30+ in first 12 months
- Went 60+ in first 12 months
- Went 90+ in fist 12 months
- Went x+ in first 18 months
- Went 30+ in first 18 months
- Went 60+ in first 18 months
- Went 90+ in fist 18 months

In [28]:
mnths = [3,6,9,12,18]
dpds = [0,30,60,90]

In [29]:
mnths = [6,12,15,18,20,24]
dpds = [0,29,59,89]

In [30]:
res_dict = {}

In [31]:
fin_df = pd.DataFrame()
fin_df['id'] = grouped_df.index
fin_df.set_index('id',inplace=True)

```python
mobs = [6,12,15,18,20,24]
dpd = [0,29,59,89]
bad_def = pd.DataFrame()
for i in mobs:
    for ii in dpd:
        t = df[df['mob'] <= i]
        t = pd.pivot_table(t,index='id',values='dpd',aggfunc='max').reset_index()
        t['proxy_bad'] = np.where(t['dpd'] > ii,1,0)
        temp = pd.merge(o_base,t[['id','proxy_bad']],how='left',)
```

In [32]:
for i in mnths:
    for ii in dpds:
        temp_df = df.drop(df[df['mob'] > i].index)
        temp_df = temp_df.groupby('id')[['dpd']].max()
        temp_df['{}+ in {} months'.format(ii,i)] = np.where(temp_df['dpd'] > ii,1,0)
        fin_df = pd.merge(fin_df,temp_df,right_index=True,left_index=True)

In [33]:
fin_df

,dpd_x,0+ in 6 months,dpd_y,29+ in 6 months,dpd_x,59+ in 6 months,dpd_y,89+ in 6 months,dpd_x,0+ in 12 months,...,dpd_y,89+ in 20 months,dpd_x,0+ in 24 months,dpd_y,29+ in 24 months,dpd_x,59+ in 24 months,dpd_y,89+ in 24 months
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,28,0,28,1,28,0,28,0,28,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
fin_df.drop(columns=[i for i in fin_df.columns if i.lower()[:3] == 'dpd'][:2],inplace=True)

In [35]:
fin_df = pd.merge(fin_df,merged_df,right_index=True,left_index=True)

In [36]:
merged_df.columns

Index(['max_dpd_x', 'eventual_bad', 'max_dpd_y', 'proxy_bad'], dtype='object')

In [37]:
fin_df.drop(columns=['max_dpd_x','max_dpd_y'],inplace=True)

In [38]:
fin_df.to_csv('Conversion Capture temp.csv')

In [39]:
# temp_df = df.drop(df[df['mob'] > 3].index)

In [40]:
fin_cols = fin_df.columns

In [41]:
precision_score(fin_df['eventual_bad'],fin_df[fin_cols[10]])

0.6079295154185022

In [42]:
for i in fin_cols:
    res_dict[i] = {'Total eventual bad':sum(fin_df['eventual_bad']),'Total proxy bad':sum(fin_df[i]),'Accuracy score':accuracy_score(fin_df['eventual_bad'],fin_df[i]),'Precision score':precision_score(fin_df['eventual_bad'],fin_df[i]),'Recall score':recall_score(fin_df['eventual_bad'],fin_df[i]),'F1 score':f1_score(fin_df['eventual_bad'],fin_df[i]),'F2_score':fbeta_score(fin_df['eventual_bad'],fin_df[i],beta=2)}

In [43]:
import json
with open('res.json','w') as js:
    json.dump(res_dict,js)

```python
import re
fin_df.rename(columns=lambda x: re.sub('\+{1}','',x),inplace=True)
```

In [44]:
fin_df

,0+ in 6 months,29+ in 6 months,59+ in 6 months,89+ in 6 months,0+ in 12 months,29+ in 12 months,59+ in 12 months,89+ in 12 months,0+ in 15 months,29+ in 15 months,...,0+ in 20 months,29+ in 20 months,59+ in 20 months,89+ in 20 months,0+ in 24 months,29+ in 24 months,59+ in 24 months,89+ in 24 months,eventual_bad,proxy_bad
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
df_fin = pd.read_json('res.json')

In [46]:
df_fin = df_fin.T

In [47]:
df_fin.head()

,Total eventual bad,Total proxy bad,Accuracy score,Precision score,Recall score,F1 score,F2_score
0+ in 6 months,234.0,150.0,0.957124,0.280000,0.179487,0.218750,0.193370
29+ in 6 months,234.0,80.0,0.967129,0.525000,0.179487,0.267516,0.206693
59+ in 6 months,234.0,0.0,0.966557,0.000000,0.000000,0.000000,0.000000
89+ in 6 months,234.0,0.0,0.966557,0.000000,0.000000,0.000000,0.000000
0+ in 12 months,234.0,370.0,0.947978,0.324324,0.512821,0.397351,0.459418


In [49]:
df_fin.sort_values(by='F2_score',ascending=False).head(15)

,Total eventual bad,Total proxy bad,Accuracy score,Precision score,Recall score,F1 score,F2_score
eventual_bad,234.0,234.0,1.000000,1.000000,1.000000,1.000000,1.000000
89+ in 24 months,234.0,234.0,1.000000,1.000000,1.000000,1.000000,1.000000
59+ in 24 months,234.0,393.0,0.977276,0.595420,1.000000,0.746411,0.880361
89+ in 20 months,234.0,184.0,0.992854,1.000000,0.786325,0.880383,0.821429
89+ in 18 months,234.0,184.0,0.992854,1.000000,0.786325,0.880383,0.821429
29+ in 24 months,234.0,526.0,0.958268,0.444867,1.000000,0.615789,0.800274
29+ in 20 months,234.0,526.0,0.958268,0.444867,1.000000,0.615789,0.800274
59+ in 18 months,234.0,273.0,0.980134,0.673993,0.786325,0.725838,0.760959
59+ in 20 months,234.0,323.0,0.972988,0.569659,0.786325,0.660682,0.730739
29+ in 18 months,234.0,362.0,0.967700,0.511050,0.790598,0.620805,0.712635


Best proxy bad definition would be <strong>90+ in 18 months</strong>